In [ ]:
'''
This example demonstrates deterministic record linkage to link two files.
In determininistic record linkage, each compared attribute get a certain
weight (coefficient). The higher the weight, the more dicriminating the
variable is. A low weight indicate a less discriminating variable. For
example, the given name has a higher weight than the hometown.
This example uses FEBRL4 datasets. These datasets contain records about
individuals.
Deterministic RL parameters are:
intercept = -11.0
coefficients = [1.5, 1.5, 8.0, 6.0, 2.5, 6.5, 5.0]
'''

In [1]:
from __future__ import print_function

import recordlinkage as rl
from recordlinkage.index import Block
from recordlinkage.compare import Exact, String

In [2]:
'''
Generated as one data set with 10000 records (5000 originals and 5000 duplicates, with one duplicate per original), 
the originals have been split from the duplicates, into dataset4a.csv (containing the 5000 original records) 
and dataset4b.csv (containing the 5000 duplicate records). 
These two data sets can be used for testing linkage procedures.
'''

from recordlinkage.datasets import load_febrl4

In [3]:
# set logging
rl.logging.set_verbosity(rl.logging.INFO)

In [4]:
# load datasets
print('Loading data...')
dfA, dfB, true_links = load_febrl4(return_links=True)
print(len(dfA), 'records in dataset A')
print(len(dfB), 'records in dataset B')
print(len(true_links), 'links between dataset A and B')

Loading data...
5000 records in dataset A
5000 records in dataset B
5000 links between dataset A and B


In [5]:
# start indexing
print('Build index...')
indexer = rl.Index()
indexer.add(Block('given_name'))
indexer.add(Block('surname'))
indexer.add(Block('soc_sec_id'))
candidate_links = indexer.index(dfA, dfB)
print(len(candidate_links), 'candidate links between dataset A and B')

Build index...
INFO:recordlinkage:indexing - initialize Index class
INFO:recordlinkage:indexing [1/?] - time: 0.83s - pairs: 160170/25000000 - rr: 0.99359
INFO:recordlinkage:indexing [1/?] - time: 0.83s - pairs_total: 160170/25000000 - rr_total: 0.99359
160170 candidate links between dataset A and B


In [6]:
# start comparing
print('Start comparing...')
comparer = rl.Compare()
comparer.add(Exact('given_name', 'given_name', label='given_name'))
comparer.add(String('surname', 'surname', method='jarowinkler',
                    threshold=0.85, label='surname'))
comparer.add(Exact('date_of_birth', 'date_of_birth', label='date_of_birth'))
comparer.add(Exact('suburb', 'suburb', label='suburb'))
comparer.add(Exact('state', 'state', label='state'))
comparer.add(String('address_1', 'address_1', threshold=0.85,
                    label='address_1'))
comparer.add(String('address_2', 'address_2', threshold=0.85,
                    label='address_2'))
features = comparer.compute(candidate_links, dfA, dfB)

print('feature shape', features.shape)

Start comparing...
INFO:recordlinkage:comparing - initialize Compare class
INFO:recordlinkage:comparing [1/?] - time: 2.92s - pairs: 160170
INFO:recordlinkage:comparing [1/?] - time: 2.92s - pairs_total: 160170
feature shape (160170, 7)


In [8]:
# use the Logistic Regression Classifier
# this classifier is equivalent to the deterministic record linkage approach
intercept = -11.0
coefficients = [1.5, 1.5, 8.0, 6.0, 2.5, 6.5, 5.0]

print('Deterministic classifier')
print('intercept', intercept)
print('coefficients', coefficients)

Deterministic classifier
intercept -11.0
coefficients [1.5, 1.5, 8.0, 6.0, 2.5, 6.5, 5.0]


In [9]:
logreg = rl.LogisticRegressionClassifier(
    coefficients=coefficients, intercept=intercept)
links = logreg.predict(features)

print(len(links), 'links/matches')

INFO:recordlinkage:Classification - predict matches and non-matches
4895 links/matches


In [10]:
# return the confusion matrix
conf_logreg = rl.confusion_matrix(true_links, links, len(candidate_links))
print('confusion matrix')
print(conf_logreg)

confusion matrix
[[  4893    107]
 [     2 155168]]


In [11]:
# compute the F-score for this classification
fscore = rl.fscore(conf_logreg)
print('fscore', fscore)
recall = rl.recall(true_links, links)
print('recall', recall)
precision = rl.precision(true_links, links)
print('precision', precision)

fscore 0.9889843355229915
recall 0.9786
precision 0.999591419816139
